In [1]:
import ortools
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
items=[
    {
    "id": "0",
    "L" : 5,
    "W" : 7,
    },
    {
    "id": "1",
    "L" : 7,
    "W" : 0,
    },
    {
    "id": "2",
    "L" : 3,
    "W" : 2,
    },
    {
    "id": "3",
    "L" : 6,
    "W" : 0,
    },
    {
    "id": "4",
    "L" : 8,
    "W" : 3,
    },
    {
    "id": "5",
    "L" : 9,
    "W" : 4,
    },
    {
    "id": "6",
    "L" : 7,
    "W" : 8,
    },
    {
        "id" : "7",
        "L" : 8,
        "W" : 9
    }
]
num_of_stock_rolls = len(items)

num_of_possible_stock_roll = 4
SR_val_max_limit = 7
SR_val_min_limit = 4


In [3]:
x={}
for i in items:
    for j in range(num_of_stock_rolls):
        x[(i["id"],j,'L')]=solver.IntVar(0,1,f'x_{i["id"]}_{j}_l')
        if i['W']!=0:
            x[(i["id"],j,'W')]=solver.IntVar(0,1,f'x_{i["id"]}_{j}_w')
#print(x)

y={}
for j in range(num_of_stock_rolls):
    y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

In [4]:
# contrain to choice either L or W
for i in items:
    i_id=i['id']
    solver.Add(sum(x[i_id,j,'L']+(x[i_id,j,'W'] if i['W']!=0 else 0)  for j in range(num_of_stock_rolls))<=1)
    
for j in range(num_of_stock_rolls):
    solver.Add(sum(x[i['id'],j,'L'] + (x[i['id'],j,'W'] if i['W']!=0 else 0)  for i in items) <=num_of_possible_stock_roll)

for j in range(num_of_stock_rolls):
    solver.Add(sum(x[i['id'],j,'L']*i['L'] + (x[i['id'],j,'W'] if i['W']!=0 else 0)*i['W']  for i in items) <=y[j]*SR_val_max_limit)

for j in range(num_of_stock_rolls):
    solver.Add(sum(x[i['id'],j,'L']*i['L'] + (x[i['id'],j,'W'] if i['W']!=0 else 0)*i['W']  for i in items) >=y[j]*SR_val_min_limit)



In [5]:
# Objective: minimize the number of bins used.
solver.Minimize(solver.Sum([y[j] for j in range(num_of_stock_rolls)]))
l=[]
#fSR_val_max_limit -  for i in items for j in range(num_of_stock_rolls)
for j in range(num_of_stock_rolls):
    remain=SR_val_max_limit
    for i in items:
        remain = remain - x[i['id'],j,'L']*i['L']+(x[i['id'],j,'W']*i['W'] if i['W']!=0 else 0)
    l.append(remain)
solver.Minimize(solver.Sum(l))
solver.Maximize(solver.Sum([x[i['id'],j,'L']+(x[i['id'],j,'W'] if i['W']!=0 else 0)   for j in range(num_of_stock_rolls) for i in items]))

In [6]:
status = solver.Solve()
print(status)

0


In [8]:
taken=[]
if status==pywraplp.Solver.OPTIMAL:
    for j in range(num_of_stock_rolls):
        wasted=SR_val_max_limit
        if y[j].solution_value()==1:
            print("Stock Roll no : ", j)
            for i in items:
                if x[i['id'],j,'L'].solution_value() ==1:
                        print('\t'*2,f"iteam - {i['id']}","L=",i["L"])
                        taken.append(i["id"])
                        wasted-=i["L"]
                if i['W']!=0:
                    if x[i['id'],j,'W'].solution_value() ==1:
                         print('\t'*2,f"iteam - {i['id']}", "W=",i['W'])
                         taken.append(i["id"])
                         wasted-=i["W"]
        print("wasted: ",wasted)

Stock Roll no :  0
		 iteam - 2 L= 3
		 iteam - 4 W= 3
wasted:  1
Stock Roll no :  1
		 iteam - 6 L= 7
wasted:  0
Stock Roll no :  2
		 iteam - 1 L= 7
wasted:  0
Stock Roll no :  3
		 iteam - 3 L= 6
wasted:  1
Stock Roll no :  4
		 iteam - 5 W= 4
wasted:  3
Stock Roll no :  5
		 iteam - 0 L= 5
wasted:  2
wasted:  7
wasted:  7


In [201]:
for i in items:
    if i['id'] not in taken:
        print(i['id'])

7
